In [ ]:

import pandas as pd
from pandas_datareader import data as wb
import openpyxl
from breeze_connect import BreezeConnect

In [ ]:
session_token = '1815459'
api_key = '6588c73p34F6^7A91VD4_3c328563!36'
api_secret = '1!3#3OLWo0R^8217y381vq75R015G806'

In [ ]:
expiry_date="2021-03-18T07:00:00.000Z"

def option_name_generator(expiry):
    a = expiry.split('T')
    date=str(a[0])
    a1 = list(map(str,date.split('-')))
    final = a1[0]+"_"+a1[1]+"_"+a1[2]+"_"+"option.xlsx"
    return final
print(option_name_generator(expiry_date))
def spot_name_generator(expiry):
    a = expiry.split('T')
    date=str(a[0])
    a1 = list(map(str,date.split('-')))
    final = a1[0]+"_"+a1[1]+"_"+a1[2]+"_"+"spot.xlsx"
    return final
def future_name_generator(expiry):
    a = expiry.split('T')
    date=str(a[0])
    a1 = list(map(str,date.split('-')))
    final = a1[0]+"_"+a1[1]+"_"+a1[2]+"_"+"future.xlsx"
    return final
print(future_name_generator(expiry_date))


In [ ]:
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret,session_token=session_token)



In [ ]:
stock_code = "CNXBAN"
interval_spot = "5minute"
interval_option = "1minute"
interval_futures = "1minute"

from_date_future = "2021-10-27T07:00:00.000Z"
to_date_future = "2021-10-28T07:00:00.000Z"

from_date_option = "2021-03-12T07:00:00.000Z"
to_date_option = "2021-03-18T07:00:00.000Z"

from_date_spot= "2021-03-12T07:00:00.000Z"
to_date_spot = "2021-03-18T07:00:00.000Z"

expiry_date="2021-03-18T07:00:00.000Z"



In [ ]:
option = pd.ExcelWriter(option_name_generator(expiry_date), engine='xlsxwriter')
spot = pd.ExcelWriter(spot_name_generator(expiry_date), engine='xlsxwriter')
futures = pd.ExcelWriter(future_name_generator(expiry_date), engine='xlsxwriter')


In [ ]:
sdata = breeze.get_historical_data(interval=interval_spot,
                                   from_date= from_date_spot,
                                   to_date= to_date_spot,
                                   stock_code=stock_code,
                                   exchange_code="NSE",
                                   product_type="cash",
                                   )
try:
    spotdata = pd.DataFrame(sdata['Success'],)
    if(spotdata.empty==True):
        print("Empty")
    else:
        spotdata.to_excel(spot, sheet_name='Spot',index=False)
        
        print(spotdata)
except:
    print('Encountered an Error!!!')
spot.save()

In [ ]:
fs=1
Expiry = ["2021-10-28T07:00:00.000Z","2021-11-25T07:00:00.000Z"]

for e in Expiry:

    fdata = breeze.get_historical_data(interval=interval_futures,
                            from_date= from_date_future,
                            to_date= to_date_future,
                            stock_code=stock_code,
                            exchange_code="NFO",
                            product_type="futures",
                            expiry_date=e,
                            right="others",
                            strike_price="0")
    futuresdata = pd.DataFrame(fdata['Success'])
    print(futuresdata)
    
    if(fs!=0):
        try:
            futuresdata.to_excel(futures, sheet_name='Futures',startrow=fs,header=False,index=False)
            fs+=futuresdata.index[-1]+1
            print(futuresdata)
        except:
            print("found an error!!!")
            continue
    
    else:
        try:

            futuresdata.to_excel(futures, sheet_name='Futures',startrow=fs,index=False,header=True)
            fs+=futuresdata.index[-1]+1
            print(futuresdata)
        except:
            print("found an error!!!")
            fs = 0
            continue

futures.save()

In [ ]:
from numpy import put
from requests import head


oc=0
op=0
Strike = ["33500","33600","33700","33800","33900","34000","34100","34200","34300","34400","34500","34600","34700","34800",\
          "34900","35000","35100","35200","35300","35400","35500","35600","35700","35800","35900","36000","36100","36200",\
          "36300","36400","36500","36600"]

for s in Strike:

    cdata = breeze.get_historical_data(interval=interval_option,
                                  from_date= from_date_option,
                                  to_date= to_date_option,
                                  stock_code=stock_code,
                                  exchange_code="NFO",
                                  product_type="options",
                                  expiry_date=expiry_date,
                                  right="call",
                                  strike_price=s)

        
    calldata = pd.DataFrame(cdata['Success'])
    if(oc!=0):
        try:
            calldata.to_excel(option, sheet_name='Call',startrow=oc,header = False, index = False)
            oc+=calldata.index[-1]+1
            print(calldata)
        except:
            print("found an error!!!")
            continue
    
    else:
        print(calldata)
        try:
            calldata.to_excel(option, sheet_name='Call',startrow=oc,index = False)
            oc+=calldata.index[-1]+1
        except:
            print("found an error!!!")
            oc = 0
            continue

    pdata = breeze.get_historical_data(interval=interval_option,
                                  from_date= from_date_option,
                                  to_date= to_date_option,
                                  stock_code=stock_code,
                                  exchange_code="NFO",
                                  product_type="options",
                                  expiry_date=expiry_date,
                                  right="put",
        
                                  strike_price=s)

    
    putdata = pd.DataFrame(pdata['Success'])
    if(op!=0):
        try:
            putdata.to_excel(option, sheet_name='Put',startrow=op,header=False,index=False )
            op+=putdata.index[-1]+1
            print(putdata)
            
        except:
            print("found an error!!!")
            continue
    else:
        try:
            print(putdata)
            putdata.to_excel(option, sheet_name='Put',startrow=op ,index = False)
            op+=putdata.index[-1]+1
        except:
            print("found an error!!!")
            op=op
            continue
    # print(str(oc).ljust(10) + "         "+str(op).rjust(10))
    
    
option.save()
